In [7]:
# these three lines swap the stdlib sqlite3 lib with the pysqlite3 package
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader # UnstructuredFileLoader는 pdf, txt, docx를 다 열 수 있음
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.vectorstores.chroma import Chroma #vector stores사용(여러 vector stores중 chroma는 오픈소스임)
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore

cache_dir = LocalFileStore("../.cache/") # embedding한 vector를 캐싱하기 위해 캐쉬 디렉토리 설정

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
) # 특정 문자열을 기준으로 끊어줌

loader = UnstructuredFileLoader("../../files/chapter_one.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

# cache embedding을 설정함으로써 embedding을 할 때, 캐시에 embedding이 이미 존재하는지 확인하고
# 없으면 vector store를 호출할때 문서들과 OpenAIEmbeddings를 사용하게 됨
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir    
)


vectorstore = Chroma.from_documents(docs, cached_embeddings)


In [8]:
""" 
 문서를 chunk로 splite한 뒤, vector store에 저장하면,
 vector store를 이용하여 질문(query)와 관련된 문서를 찾을 수 있음
 만약 받은 문서들이 너무 크다면 엄청 큰 문서가 LLM에 전달될거고 그 만큼 많은 돈을 지출할 것이므로
 작게 분할하는게 좋음 (그러나 너무 작으면 문맥의 의미가 사라지므로 안 좋음)
"""
results = vectorstore.similarity_search("where does winston live")

len(results)

4